**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Jul 26 15:55:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
 Classroom  'Colab Notebooks'  'Files to share'  'My documents'   yolov3


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 14073 (delta 34), reused 28 (delta 13), pack-reused 14000
Receiving objects: 100% (14073/14073), 12.83 MiB | 3.83 MiB/s, done.
Resolving deltas: 100% (9557/9557), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%%capture
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
# !make

In [ ]:
!make

**3) Configure Darknet network for training YOLO V3**

In [6]:
!cp cfg/yolov3-tiny.cfg cfg/yolov3-tiny_training.cfg

In [7]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3-tiny_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3-tiny_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3-tiny_training.cfg

!sed -i '135 s@classes=80@classes=2@' cfg/yolov3-tiny_training.cfg
!sed -i '177 s@classes=80@classes=2@' cfg/yolov3-tiny_training.cfg

!sed -i '127 s@filters=255@filters=21@' cfg/yolov3-tiny_training.cfg
!sed -i '171 s@filters=255@filters=21@' cfg/yolov3-tiny_training.cfg

#(X+5)*3

In [ ]:
# Create folder on google drive so that we can save there the weights
# !mkdir "/mydrive/yolov3"

In [8]:
!echo -e 'ok\ncall' > data/obj.names
!echo -e 'classes= 2\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
# !wget https://pjreddie.com/media/files/darknet53.conv.74
!cp /mydrive/yolov3/yolov3-tiny.conv.11 /content/darknet/

**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

In [11]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/033.jpg', 'data/obj/018.jpg', 'data/obj/032.jpg', 'data/obj/144.jpg', 'data/obj/036.jpg', 'data/obj/160.jpg', 'data/obj/045.jpg', 'data/obj/128.jpg', 'data/obj/166.jpg', 'data/obj/014.jpg', 'data/obj/048.jpg', 'data/obj/124.jpg', 'data/obj/138.jpg', 'data/obj/013.jpg', 'data/obj/026.jpg', 'data/obj/060.jpg', 'data/obj/01.jpg', 'data/obj/09.jpg', 'data/obj/016.jpg', 'data/obj/131.jpg', 'data/obj/148.jpg', 'data/obj/120.jpg', 'data/obj/134.jpg', 'data/obj/064.jpg', 'data/obj/11.jpg', 'data/obj/16.jpg', 'data/obj/151.jpg', 'data/obj/065.jpg', 'data/obj/015.jpg', 'data/obj/058.jpg', 'data/obj/117.jpg', 'data/obj/017.jpg', 'data/obj/155.jpg', 'data/obj/163.jpg', 'data/obj/052.jpg', 'data/obj/18.jpg', 'data/obj/111.jpg', 'data/obj/164.jpg', 'data/obj/044.jpg', 'data/obj/145.jpg', 'data/obj/130.jpg', 'data/obj/113.jpg', 'data/obj/056.jpg', 'data/obj/162.jpg', 'data/obj/119.jpg', 'data/obj/068.jpg', 'data/obj/114.jpg', 'data/obj/110.jpg', 'data/obj/157.jpg', 'data/obj/035.jpg', 'dat

In [12]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

**Start time : 2:44 AM**

In [14]:
%%capture
# Start the training
!./darknet detector train data/obj.data cfg/yolov3-tiny_training.cfg yolov3-tiny.conv.11 -dont_show